In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
# Train/Test Data
train = pd.read_csv('/content/drive/MyDrive/Data Science/Digit Recognizer/Data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data Science/Digit Recognizer/Data/test.csv')

In [ ]:
# Separate training data into data/labels
X_train = train.iloc[:, 1:]
y_train = train.iloc[:, :1]
X_test = test

# Convert into PyTorch tensors
X_train = torch.Tensor(X_train.values).to(device)
y_train = torch.Tensor(y_train.values).to(device)
X_test = torch.Tensor(X_test.values).to(device)

In [ ]:
# Create model
class Linear(nn.Module):
  def __init__(self):
    super(Linear, self).__init__()
    self.fc1 = nn.Linear(784, 128)
    self.bn1 = nn.BatchNorm1d(128)
    self.dropout1 = nn.Dropout(0.2)

    self.fc2 = nn.Linear(128, 64)
    self.bn2 = nn.BatchNorm1d(64)
    self.dropout2 = nn.Dropout(0.2)

    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    # x = self.bn1(x)
    # x = self.dropout1(x)

    x = F.relu(self.fc2(x))
    # x = self.bn2(x)
    # x = self.dropout2(x)

    x = self.fc3(x)
    return x

In [ ]:
# Create model and optimizer
model = Linear().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train in batches (30)
batch_size = 30

for epoch in range(20):
  for i in range(0, len(X_train), batch_size):
    optimizer.zero_grad()
    outputs = model(X_train[i:i+batch_size]).to(device)
    target = y_train[i:i+batch_size].squeeze().long().to(device)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 20, loss.item()))

Epoch [1/20], Loss: 0.0411
Epoch [2/20], Loss: 0.0068
Epoch [3/20], Loss: 0.0354
Epoch [4/20], Loss: 0.0359
Epoch [5/20], Loss: 0.0595
Epoch [6/20], Loss: 0.0310
Epoch [7/20], Loss: 0.0121
Epoch [8/20], Loss: 0.0048
Epoch [9/20], Loss: 0.0404
Epoch [10/20], Loss: 0.1104
Epoch [11/20], Loss: 0.1517
Epoch [12/20], Loss: 0.0070
Epoch [13/20], Loss: 0.0000
Epoch [14/20], Loss: 0.0056
Epoch [15/20], Loss: 0.0008
Epoch [16/20], Loss: 0.0003
Epoch [17/20], Loss: 0.0054
Epoch [18/20], Loss: 0.0012
Epoch [19/20], Loss: 0.0000
Epoch [20/20], Loss: 0.0004


In [ ]:
# Apply model to test data
test_output = model(X_test)

# Argmax the predictions
predictions = torch.argmax(test_output, dim=1)
predictions = predictions.cpu().detach().numpy()

In [ ]:
# Create CSV file for submission
submission = pd.DataFrame({'ImageId': range(1, len(predictions)+1), 'Label': predictions})
submission.to_csv('submission.csv', index=False)

# Download
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>